In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
from pandas.io.json import json_normalize
from __future__ import (absolute_import, division, print_function)
import os
import json
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from geopandas import GeoSeries, GeoDataFrame

data_path = "./data"
%matplotlib inline


In [2]:
import psycopg2
import psycopg2.extras

def pgconnect(credential_filepath):
    try:
        with open(credential_filepath) as f:
            db_conn_dict = json.load(f)
        conn = psycopg2.connect(**db_conn_dict)
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn

# please update the details in data2x01_db.json
credfilepath = os.path.join(data_path, "data2x01_db.json")
# credfilepath = os.path.join(data_path, "data2001_dbadmin.json")
# credfilepath = os.path.join(data_path, "data2001_db95.json")
conn = pgconnect(credfilepath)

connected


In [3]:
def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None:
                    if msg: print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

Create shape file table

In [14]:
sa2 = gpd.read_file( os.path.join(data_path, "SA2_2016_AUST.shp") )
sa2 = sa2.drop(columns = ['SA2_5DIG16', 'SA2_NAME16', 'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16', 'GCC_CODE16', 'GCC_NAME16', 'STE_CODE16', 'STE_NAME16'])
sa2

,SA2_MAIN16,AREASQKM16,geometry
0,101021007,3418.3525,POLYGON ((149.5842329970001 -35.44426999899997...
1,101021008,6.9825,POLYGON ((149.2189819130001 -35.36739402599994...
2,101021009,4.7634,POLYGON ((149.2132479820001 -35.34324799199993...
3,101021010,13.0034,"POLYGON ((149.240332114 -35.34782261699996, 14..."
4,101021011,3054.4099,"POLYGON ((149.235800967 -35.38737530299994, 14..."
...,...,...,...
2305,901021002,13.7163,(POLYGON ((96.83046599400006 -12.1763599599999...
2306,901031003,67.8134,(POLYGON ((150.6956629970001 -35.1829670079999...
2307,901041004,38.6509,(POLYGON ((167.9947210150001 -29.0453360049999...
2308,997979799,0.0000,None


In [17]:
pgquery(conn, "DROP TABLE IF EXISTS AUS", msg="cleared old table")
AUS_schema = '''CREATE TABLE AUS (
                     SA2_MAIN16 NUMERIC,
                     AREASQKM16 NUMERIC,
                     geometry GEOMETRY(MULTIPOLYGON,4326))''' 
pgquery(conn, AUS_schema, msg="create new table")

success: cleared old table
success: create new table


In [20]:
insert_stmt = """INSERT INTO AUS VALUES ( %(SA2_MAIN16)s,  %(AREASQKM16)s, ST_Multi(ST_GeomFromText(%(geom_wkt)s, 4326) ))"""

sa2['geom_wkt'] = sa2['geometry'].apply(lambda x: x.wkt if x is not None else x)

for idx, row in sa2.iterrows():
    pgquery(conn, insert_stmt, args=row, msg="inserted")


success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: i

success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: i

success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: i

success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: i

success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: inserted
success: i

In [21]:
pd.read_sql_query("select * from AUS", conn)

,sa2_main16,areasqkm16,geometry
0,101021007.0,3418.3525,0106000020E6100000010000000103000000010000005F...
1,101021008.0,6.9825,0106000020E6100000010000000103000000010000002C...
2,101041021.0,24.2329,0106000020E6100000020000000103000000010000007F...
3,101041022.0,2781.9988,0106000020E61000000100000001030000000100000083...
4,101041023.0,94.8138,0106000020E61000000A00000001030000000100000007...
...,...,...,...
2305,901021002.0,13.7163,0106000020E6100000380000000103000000010000000E...
2306,901031003.0,67.8134,0106000020E61000000300000001030000000100000017...
2307,901041004.0,38.6509,0106000020E61000001F0000000103000000010000000B...
2308,997979799.0,0.0000,None


Create Testsites Table

In [4]:
pgquery(conn, "DROP TABLE IF EXISTS TESTSITES", msg="cleared old table")
testsites_schema = """CREATE TABLE IF NOT EXISTS TESTSITES (
                         site_id INTEGER PRIMARY KEY,
                         longitude FLOAT,
                         latitude FLOAT,
                         test_capacity INTEGER
                         
                   )"""
pgquery(conn, testsites_schema, msg="create new table")


success: cleared old table
success: create new table


In [5]:
testsites_data = pd.read_csv('covid19_nsw_testsites_simulated_capacity_modified.csv')
table_name = "testsites"
testsites_data.to_sql(table_name, con=conn, if_exists='append')

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': relation "sqlite_master" does not exist
LINE 1: SELECT name FROM sqlite_master WHERE type='table' AND name=?...
                         ^


In [ ]:
pd.read_sql_query("select * from testsites", conn)

Create Postcodes Table

In [ ]:
pgquery(conn, "DROP TABLE IF EXISTS postcodes", msg="cleared old table")
postcodes_schema = """CREATE TABLE IF NOT EXISTS POSTCODES (
                         id INTEGER PRIMARY KEY,
                         postcode INTEGER,
                         locality VARCHAR(40),
                         longitude FLOAT,
                         latitude FLOAT
                         
                   )"""
pgquery(conn, postcodes_schema, msg="create new table")

In [ ]:
postcodes_data = pd.read_csv('NSW_Postcodes.csv', index_col=0)
table_name = "postcodes"
postcodes_data.to_sql(table_name, con=conn, if_exists='append')

Create Healthservice Table

In [15]:
conn.execute("DROP TABLE IF EXISTS HEALTHSERVICE")
healthservice_schema = """CREATE TABLE IF NOT EXISTS HEALTHSERVICE (
                         id INTEGER,
                         name VARCHAR(100),
                         category VARCHAR(50),
                         num_beds FLOAT,
                         suburb VARCHAR(60),
                         state CHAR(3),
                         postcode INTEGER,
                         longitude FLOAT,
                         latitude FLOAT,
                         PRIMARY KEY(id, name)
                         
                   )"""
conn.execute(healthservice_schema)

AttributeError: 'psycopg2.extensions.connection' object has no attribute 'execute'

In [ ]:
healthservice_data = pd.read_csv('HealthServices_modified.csv', index_col=0)
table_name = "healthservice"
healthservice_data.to_sql(table_name, con=conn, if_exists='append')

Create Population Table

In [ ]:
conn.execute("DROP TABLE IF EXISTS POPULATION")
population_schema = """CREATE TABLE IF NOT EXISTS Population (
                         area_id INTEGER PRIMARY KEY,
                         area_name VARCHAR(100),
                         total_persons INTEGER,
                         females INTEGER,
                         males INTEGER
                         
                   )"""
conn.execute(population_schema)

In [ ]:
population_data = pd.read_csv('PopulationStats2016.csv', index_col=['area_id'], usecols=['area_id', 'area_name', 'total_persons', 'females', 'males'])
table_name = "population"
population_data.to_sql(table_name, con=conn, if_exists='append')

Create Neighbourhoods Table

In [ ]:
conn.execute("DROP TABLE NEIGHBOURHOODS")
neighbourhood_schema = """CREATE TABLE IF NOT EXISTS NEIGHBOURHOODS (
                         area_id INTEGER PRIMARY KEY NOT NULL,
                         area_name VARCHAR(100),
                         land_area FLOAT,
                         population INTEGER,
                         number_of_dwellings INTEGER,
                         number_of_businesses INTEGER,
                         median_annual_household_income INTEGER,
                         avg_monthly_rent INTEGER
                         
                   )"""
conn.execute(neighbourhood_schema)

In [ ]:
neighbourhoods_data = pd.read_csv('Neighbourhoods.csv', index_col=0)
neighbourhoods_data.dropna()
table_name = "neighbourhoods"
neighbourhoods_data.to_sql(table_name, con=conn, if_exists='append')

Create Areas Table

In [ ]:
conn.execute("DROP TABLE IF EXISTS AREAS")
areas_schema = """CREATE TABLE IF NOT EXISTS areas (
                         area_id INTEGER PRIMARY KEY,
                         area_name VARCHAR(100),
                         parent_area_id INTEGER
                         
                   )"""
conn.execute(areas_schema)

In [ ]:
areas_data = pd.read_csv('StatisticalAreas.csv', index_col=0)
table_name = "areas"
areas_data.to_sql(table_name, con=conn, if_exists='append')

In [ ]:
population_stats_schema = """
CREATE TABLE IF NOT EXISTS area (
area_id INTEGER PRIMARY KEY,
area_name VARCHAR(100),
total_persons INTEGER,

)

"""

 Create Traffic Table (additional JSON dataset)

In [ ]:
filename = "Traffic_Volume_Viewer_-_2020_Data.geojson"

In [ ]:
traffic_data = gpd.read_file(filename)
traffic_data.drop(columns=['road_name', 'cardinal_direction_name', 'classification_type', 'year', 'period', 'wgs84_latitude', 'wgs84_longitude'])

In [ ]:
pgquery(conn, "DROP TABLE IF EXISTS TRAFFIC", msg="cleared old table")
traffic_schema = """CREATE TABLE IF NOT EXISTS TRAFFIC (
                         station_id VARCHAR(100),
                         suburb VARCHAR(100),
                         traffic_count INTEGER,
                         geometry GEOMETRY(POINT, 4326)
                   )"""
pgquery(conn, traffic_schema, msg="create new table")

In [ ]:
insert_stmt = """INSERT INTO Traffic VALUES ( %(station_id)s, %(suburb)s, %(traffic_count)s,
                                            ST_PointFromText(%(geom_wkt)s, 4326) )"""

traffic_data['geom_wkt'] = traffic_data['geometry'].apply(lambda x: x.wkt)

for idx, row in traffic_data.iterrows():
    pgquery(conn, insert_stmt, args=row, msg="inserted")

In [ ]:
query = "SELECT COUNT(*) FROM TRAFFIC"
result= pgquery(conn, query, returntype='dict')
from pprint import pprint 
print(result)

In [ ]:
pd.read_sql_query("SELECT * FROM TRAFFIC", conn)

Population_density Calculation

In [ ]:
conn.execute("ALTER TABLE neighbourhoods ADD IF NOT EXISTS population_density FLOAT;")
calculate = """INSERT INTO neighbourhoods(population_density)
               SELECT population/land_area
               FROM neighbourhoods
            """

In [ ]:
pd.read_sql_query(calculate, conn)

In [ ]:
table_name = "neighbourhoods"
df.to_sql(table_name, con=conn, if_exists='append')

In [ ]:
pd.read_sql_query("select * from neighbourhoods", conn)

Population_age Calculation

Healthservice_density Calculation

Hospitalbed_density Calculation

Traffic_volume Calculation

In [ ]:
conn.close()
db.dispose()